# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


first_region = pd.read_csv('/datasets/geo_data_0.csv')
second_region = pd.read_csv('/datasets/geo_data_1.csv')
third_region = pd.read_csv('/datasets/geo_data_2.csv')

regions = [first_region, second_region, third_region]

def display_regions():
    for i in range(len(regions)):
        print(i+1, 'РЕГИОН')
        display(regions[i].head(5))
        regions[i].info()
        print()
        
display_regions()        

1 РЕГИОН


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

2 РЕГИОН


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

3 РЕГИОН


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



In [2]:
for r in regions:
    print()
    print(r.describe())


                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%     

In [3]:


for r in regions:
    display(r[r['product']==0])

,id,f0,f1,f2,product
57263,zCM5W,-0.702064,0.375992,0.236572,0.0


,id,f0,f1,f2,product
11,OXyvW,16.320755,-0.562946,-0.001783,0.0
13,igmai,6.695604,-0.749449,-0.007630,0.0
62,Qjy5w,21.418478,-5.134490,-0.002836,0.0
63,G6WCj,6.822701,3.104979,-0.000723,0.0
77,MzQhL,6.750150,-11.893512,-0.001601,0.0
...,...,...,...,...,...
99936,YrRU8,5.085749,-3.980305,0.005063,0.0
99948,Jbnur,8.277805,-9.178818,0.003275,0.0
99956,aV1cJ,13.343983,-1.290200,0.005980,0.0
99961,Zjbn2,13.854163,-11.528089,-0.005556,0.0


,id,f0,f1,f2,product
68149,qeefd,-0.865596,-1.615247,-4.126441,0.0


Во втором регионе имеем нулевые значения в столбце `product`


In [4]:
len(*np.where(second_region['product']==0))

8235

In [5]:
# Нулевые строки в product - удалены
second_region = second_region.drop(*np.where(second_region['product']==0))
second_region


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [6]:
third_region.duplicated().sum()

0

# Вывод (шаг 1)

Видим, что данные в таблицах однородны, т.е столбцы, типы, форма датасетов совпадают. Дубликаты отсутсвуют. Есть нулевые значения во 2ом регионе в столбце `product`, убираем эти значения.

## Обучение и проверка модели

In [7]:
# Сделаем разбивку наших датасетов на тренировочные и валидационные

r1_target = first_region['product']
r1_features = first_region.drop(columns=['product', 'id'])

r2_target = second_region['product']
r2_features = second_region.drop(columns=['product', 'id'])

r3_target = third_region['product']
r3_features = third_region.drop(columns=['product', 'id'])


# Делим данные для каждого региона

r1_features_train, r1_features_valid, r1_target_train, r1_target_valid = train_test_split(
    r1_features, r1_target, test_size=0.25
)


r2_features_train, r2_features_valid, r2_target_train, r2_target_valid = train_test_split(
    r2_features, r2_target, test_size=0.25
)


r3_features_train, r3_features_valid, r3_target_train, r3_target_valid = train_test_split(
    r3_features, r3_target, test_size=0.25
)

In [8]:
# Обучим модель линейной регресси для кажого региона

r1_model = LinearRegression().fit(r1_features_train, r1_target_train)
r2_model = LinearRegression().fit(r2_features_train, r2_target_train)
r3_model = LinearRegression().fit(r3_features_train, r3_target_train)

r1_model_pred = r1_model.predict(r1_features_valid)
r2_model_pred = r2_model.predict(r2_features_valid)
r3_model_pred = r3_model.predict(r3_features_valid)

# Напечатаем на экране средний запас предсказанного сырья для каждого региона и RMSE модели


region_preds = [
    r1_model_pred,
    r2_model_pred,
    r3_model_pred
]


region_answers = [
    r1_target_valid,
    r2_target_valid,
    r3_target_valid
]

for i in range(len(region_preds)):
    print('Предсказанный средний запас для', i+1, 'региона:', region_preds[i].mean())
    print('Реальный средний запас для', i+1, 'региона:', region_answers[i].mean())
    print('RMSE:', mean_squared_error(region_answers[i], region_preds[i])**0.5)
    print()


Предсказанный средний запас для 1 региона: 92.57186128012357
Реальный средний запас для 1 региона: 92.38626425277683
RMSE: 37.78197666823993

Предсказанный средний запас для 2 региона: 75.12165895533172
Реальный средний запас для 2 региона: 75.11777433995009
RMSE: 0.888107729150056

Предсказанный средний запас для 3 региона: 94.89315060089068
Реальный средний запас для 3 региона: 95.65176551668097
RMSE: 39.9767516257672



 Линейные модели любят, когда данные масштабированы.


# Вывод (шаг 2)

В ходе предсказаний моделей, средние значения на трейне/валиде (для всех регионов) не сильно различны. Больший RMSE получился у 3го региона, значит на этом регионе модель имеет тенденцию ошибаться. Но и ср.значение запасов в этом регионе больше остальных.

In [9]:
len(first_region['id'].unique())

99990

## Подготовка к расчёту прибыли

Рассчитаем нужный объём сырья для безубыточной разработки новой скважины, и сравним зполученное значение со средними показателями по регионам


In [10]:
TOP_OILS_WELL = 200
BUDGET = 10e9
UNIT_INCOME = 450e3
SAMP_WELLS = 500

EXPECTED_VOL = BUDGET/ (TOP_OILS_WELL * UNIT_INCOME)
print('Минимальный объём продукта с одной скважины для безубыточной разработки: {:.2f}'.format(EXPECTED_VOL))


Минимальный объём продукта с одной скважины для безубыточной разработки: 111.11


# Вывод(шаг 3)

 Расчёты показали, что для безубыточной разработки нам потребуется 111.11 ед. сырья. Также значение превышает средние показатели по всем регионам.

## Расчёт прибыли и рисков 

### Расчёт прибыли

In [11]:
r1_target_valid = r1_target_valid.reset_index(drop=True)
r2_target_valid = r2_target_valid.reset_index(drop=True)
r3_target_valid = r3_target_valid.reset_index(drop=True)


In [12]:
r1_model_pred = pd.Series(r1_model_pred)
r2_model_pred = pd.Series(r2_model_pred)
r3_model_pred = pd.Series(r3_model_pred)



In [13]:
def show_predicted_income(region, top_wells_mean, volume, income):
    print(region)
    print('Средний запас сырья среди скважин с максимальным показателем: {:.2f}'.format(top_wells_mean))
    print('Суммарный целевой объём сырья: {:.2f}'.format(volume))
    print('Прибыль для полученного объёма сырья: {:.2f}'.format(income))

In [37]:
state = 12345

# Берём выборку размера SAMP_WELLS, выбираем из неё по условию топ 200 и считаем ключевые показатели
def predicted_income(target, predicts, region, state, replace=False, show_res=True, return_res=False):
    sample_preds = predicts.sample(n=SAMP_WELLS,replace=replace,random_state=state)
    top_preds = sample_preds.sort_values(ascending=False)[:TOP_OILS_WELL]
    top_targets = target[top_preds.index]
    top_wells_mean = top_targets.mean()
    volume = sum(top_targets)
    income = volume * UNIT_INCOME - BUDGET
    if show_res:
        show_predicted_income(region, top_wells_mean, volume, income)
    if return_res:
        return income

In [38]:
predicted_income(r1_target_valid, r1_model_pred, 'Регион #1', state)

Регион #1
Средний запас сырья среди скважин с максимальным показателем: 113.53
Суммарный целевой объём сырья: 22706.17
Прибыль для полученного объёма сырья: 217775916.63


In [39]:
predicted_income(r2_target_valid, r2_model_pred, 'Регион #2', state)

Регион #2
Средний запас сырья среди скважин с максимальным показателем: 117.05
Суммарный целевой объём сырья: 23410.63
Прибыль для полученного объёма сырья: 534785740.57


In [40]:
predicted_income(r3_target_valid, r3_model_pred, 'Регион #3', state)

Регион #3
Средний запас сырья среди скважин с максимальным показателем: 117.76
Суммарный целевой объём сырья: 23551.71
Прибыль для полученного объёма сырья: 598271568.29


### Расчёт рисков

С помощью Bootstrap'a делаем 1000 выборок, и таким образом находим рапсределение прибыли. Далее ищем среднюю прибыль - 95% интервал для неё и оцениваем риски.

In [41]:
def show_risks(region, income_mean, conf_int_left, conf_int_right, loss_rate):
    print(region)
    print('Средняя прибыль {:.2f} (млн. Р)'.format(income_mean / 10**6))
    print('95% доверительный интервал: {:.2f} : {:.2f} (млн. Р)'.format(conf_int_left / 10**6, conf_int_right / 10**6))
    print('Риск убытков: {:.2f} %'.format(loss_rate * 100))

In [42]:
# C помощью техники Bootstrap оценим прибыль и риски
def risk_calc(target, predicts, region):
    """
    in: predicts - предсказания линейной модели для объёма сырья
    descr:  - функция реализует технику bootstrap, количество выборок = bootstrap_samples.
                  - Функция берёт выборку размера SAMP_WELLS, выбирает TOP_WELLS наивысших значений,
                    считает общую прибыль income_pred и сохраняет её в списке incomes.
                  - Функция выводит:  оценку средней прибыли
                                      95% доверительный интервал (считается с помощью функции quantile)
                                      риск убытков 
    """
    bootstrap_samples = 1000
    alpha = 0.05
    incomes = []
    state = np.random.RandomState(12345)
    for _ in range(bootstrap_samples):
        income = predicted_income(target, predicts, region, state, replace=True, show_res=False, return_res=True)
        incomes.append(income)

    incomes = pd.Series(incomes)
    income_mean = incomes.mean()
    conf_int_left = incomes.quantile(alpha/2)
    conf_int_right = incomes.quantile(1 - alpha/2)
    
    loss_count = 0
    for inc in incomes :
        if inc < 0 :
            loss_count += 1
    loss_rate = loss_count / bootstrap_samples
    
    
    show_risks(region, income_mean, conf_int_left, conf_int_right, loss_rate)

In [43]:
risk_calc(r1_target_valid, r1_model_pred, 'Регион 1')

Регион 1
Средняя прибыль 415.02 (млн. Р)
95% доверительный интервал: -99.53 : 912.99 (млн. Р)
Риск убытков: 5.70 %


In [44]:
risk_calc(r2_target_valid, r2_model_pred, 'Регион 2')

Регион 2
Средняя прибыль 735.18 (млн. Р)
95% доверительный интервал: 338.59 : 1131.98 (млн. Р)
Риск убытков: 0.00 %


In [45]:
risk_calc(r3_target_valid, r3_model_pred, 'Регион 3')

Регион 3
Средняя прибыль 425.74 (млн. Р)
95% доверительный интервал: -97.35 : 953.66 (млн. Р)
Риск убытков: 4.60 %


# Вывод (шаг 4)

 Для 200 наилучших скважин из 500- средний запас для всех регионов обошёл минимально необходимый объём 111,11, в то время как средние значения по всей выборке до него недотягивают. После Бутсрапирования оценили прибыли и риски по каждым регионам. Максимальная прибыль вышла у 2го региона - 735.18 млн руб. Также у данного региона самые минимальные показатели рисков - 0%, поэтому в рекоммендации идёт 2ой регион.